# Aprentissage supervisé

* 1: [Loading the database](#step1)
* 2: [Calculation of all the image's signature](#step2)
* 3: [Calculation of the centers of each class(boundaries)](#step3)
* 4: [Testing the classifier](#step4)

<a id="step1"></a>
## Loading the database 

#### For the classifier: we will use 700 images for training and 210 for testing and cross-k validation after, saving each metric calculated
#### For the neural network: We will use 700 images for training, 105 for validation and 105 for testing. 

In [1]:
# function to read all the database files and its expected classification
# input: path to the csv file
# output: a list with all the images filenames
def load_database(path):
    file_names = []
    targets = []
    f = open(path, 'r')
    
    #split all the lines into a list
    list_files = f.read().split('\n')
    
    #scan all these lines
    for files in list_files:
        #split each line with a space, we get = [filename, target]
        full_line = files.split()
        
        if(len(full_line) != 0):   
            file_names.append(full_line[0])
            targets.append(int(full_line[1]))
            #print(full_line[0], full_line[1], i)
    
    f.close()
    return file_names, targets

In [2]:
import cv2
import tree_utils as tu
import numpy as np

def calculateSignatures():
    filenames, targets = load_database('../FruitLearning/Resources/targets.csv')
    
    signatures = []
    for file in filenames:#tqdm(filenames):#filenames:
        #print(file)
        img = cv2.imread(file)
        if(img is None):
            print(file)
        else:
            #sig_1 = tu.get_hist(tu.to_HSV(img))
            sig_1 = tu.get_hist(img)
            sig_2 = tu.get_LBP(tu.get_Luminance(img))
            signatures.append([sig_1, sig_2])
    
    return signatures

In [3]:
signatures = calculateSignatures()

../FruitLearning/Resources/database/spring/137.png
../FruitLearning/Resources/database/winter/726.png
../FruitLearning/Resources/database/spring/175.png
../FruitLearning/Resources/database/summer/406.png
../FruitLearning/Resources/database/autumn/622.png
../FruitLearning/Resources/database/spring/151.png
../FruitLearning/Resources/database/autumn/533.png
../FruitLearning/Resources/database/summer/396.png
../FruitLearning/Resources/database/autumn/575.png
../FruitLearning/Resources/database/autumn/492.png
../FruitLearning/Resources/database/spring/64.png
../FruitLearning/Resources/database/spring/221.png
../FruitLearning/Resources/database/spring/228.png
../FruitLearning/Resources/database/spring/69.png
../FruitLearning/Resources/database/winter/811.png
../FruitLearning/Resources/database/autumn/671.png
../FruitLearning/Resources/database/spring/123.png
../FruitLearning/Resources/database/spring/161.png
../FruitLearning/Resources/database/winter/889.png
../FruitLearning/Resources/databa

In [ ]:
def classifieur(signatures, sample, targets):
    configuration = tu.get_nn_configuration(len(signatures))
    
    coef = [0.7, 0.3]
    min_dist = 1000
    min_index = -1
    sig_1 = tu.get_hist(sample)
    sig_2 = tu.get_LBP(tu.get_Luminance(sample))
    sample_signature = [sig_1, sig_2]
    
    for i, sig in enumerate(signatures):

        dist = tu.distance(sig, sample_signature, coef)
        if(dist < min_dist):
            min_dist = dist
            min_index = i

    
    return targets[min_index]
    

In [2]:
filenames, targets = load_database("../FruitLearning/Resources/targets.csv")
print("Number of images: ", len(filenames))

Number of images:  910


<a id='step2'></a>
## Calculation of all the image's signature

In [31]:
import cv2
import tree_utils as tu
from tqdm import tqdm

signatures = []
for file in filenames:#tqdm(filenames):#filenames:
    #print(file)
    img = cv2.imread(file)
    if(img is None):
        print(file)
    else:
        #sig_1 = tu.get_hist(tu.to_HSV(img))
        sig_1 = tu.get_hist(img)
        sig_2 = tu.get_LBP(tu.get_Luminance(img))
        signatures.append([sig_1, sig_2])

### Evaluation 

In [16]:
configuration = tu.get_nn_configuration(len(filenames))
print(configuration[0][0][0])
print(configuration[0][1])

0
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221, 

In [19]:
import numpy as np
coef = [0.7, 0.3]
confusion_matrix = np.zeros((4,4))
right = 0
for conf in configuration:
    test = conf[0][0]
    train = conf[1]
    
    min_dist = 1000
    min_index = -1
    
    for sample in train:
            dist = tu.distance(signatures[test], signatures[sample], coef)
            if(dist < min_dist):
                min_dist = dist
                min_index = sample
    
    prediction = targets[min_index]
    
    if(prediction == targets[test]):
        right += 1
    confusion_matrix[targets[test]][prediction] += 1

In [21]:
print('accuracy: ', right / len(signatures))
print(confusion_matrix)

accuracy:  0.8362637362637363
[[187.  25.   3.  31.]
 [ 27. 196.   5.   4.]
 [  9.   6. 188.   5.]
 [ 25.   6.   3. 190.]]


In [26]:
import numpy as np
coef = [0.7, 0.3]
confusion_matrix = np.zeros((4,4))
right = 0
for conf in configuration:
    test = conf[0][0]
    train = conf[1]
    
    min_dist = 1000
    min_index = -1
    
    for sample in train:
            dist = tu.distance(signatures[test], signatures[sample], coef)
            if(dist < min_dist):
                min_dist = dist
                min_index = sample
    
    prediction = targets[min_index]
    
    if(prediction == targets[test]):
        right += 1
    confusion_matrix[targets[test]][prediction] += 1

In [27]:
print('accuracy: ', right / len(signatures))
print(confusion_matrix)

accuracy:  0.8362637362637363
[[187.  25.   3.  31.]
 [ 27. 196.   5.   4.]
 [  9.   6. 188.   5.]
 [ 25.   6.   3. 190.]]


In [29]:
#with HSV
import numpy as np
coef = [0.7, 0.3]
confusion_matrix = np.zeros((4,4))
right = 0
for conf in configuration:
    test = conf[0][0]
    train = conf[1]
    
    min_dist = 1000
    min_index = -1
    
    for sample in train:
            dist = tu.distance(signatures[test], signatures[sample], coef)
            if(dist < min_dist):
                min_dist = dist
                min_index = sample
    
    prediction = targets[min_index]
    
    if(prediction == targets[test]):
        right += 1
    confusion_matrix[targets[test]][prediction] += 1

In [30]:
print('accuracy: ', right / len(signatures))
print(confusion_matrix)

accuracy:  0.7758241758241758
[[188.  24.   7.  27.]
 [ 23. 173.  28.   8.]
 [ 16.  33. 154.   5.]
 [ 21.   9.   3. 191.]]


In [ ]:
centroids -> hsv et rgb
1-ppv -> hsv et rgb
5-ppv -> hsv et rgb

In [43]:
print(signatures[0][1].shape)

(1, 64)


In [33]:
import numpy as np
def signature_mean(list_sig):
    sig_hist = [sig[0] for sig in list_sig]
    sig_lbp = [sig[1] for sig in list_sig]
    
    return [np.mean(np.array(sig_hist), axis=0), np.mean(np.array(sig_lbp), axis=0)]
        

In [5]:
print(len(signatures))

910


In [34]:
all_index_spring_sig = [ i for i,f in enumerate(filenames) if 'spring' in f]
all_spring_sig = [signatures[index] for index in all_index_spring_sig]
print(len(all_spring_sig))

all_index_winter_sig = [i for i,f in enumerate(filenames) if 'winter' in f]
all_winter_sig = [signatures[index] for index in all_index_winter_sig]
print(len(all_winter_sig))

all_index_autumn_sig = [i for i,f in enumerate(filenames) if 'autumn' in f]
all_autumn_sig = [signatures[index] for index in all_index_autumn_sig]

all_index_summer_sig = [i for i,f in enumerate(filenames) if 'summer' in f]
all_summer_sig = [signatures[index] for index in all_index_summer_sig]

246
224


In [35]:
centroid_spring = signature_mean(all_spring_sig)
centroid_winter = signature_mean(all_winter_sig)
centroid_autumn = signature_mean(all_autumn_sig)
centroid_summer = signature_mean(all_summer_sig)

In [39]:
right = 0
coef = [0.7, 0.3]
confusion_matrix = np.zeros((4,4))
for i,sig in enumerate(signatures):
    dist_spring = tu.distance(centroid_spring, sig, coef)
    dist_winter = tu.distance(centroid_winter, sig, coef)
    dist_autumn = tu.distance(centroid_autumn, sig, coef)
    dist_summer = tu.distance(centroid_summer, sig, coef)
    #prediction spring = 0, summer = 1, autumn = 2, winter = 3
    
    if(dist_spring <= dist_winter and dist_spring <= dist_autumn and dist_spring <= dist_summer):
        prediction = 0
    elif(dist_summer <= dist_winter and dist_summer <= dist_spring and dist_summer <= dist_autumn):
        prediction = 1
    elif(dist_autumn <= dist_winter and dist_autumn <= dist_spring and dist_autumn <= dist_summer):
        prediction = 2
    else:
        prediction = 3
    
    #check if prediction was good
    if(prediction == targets[i]):
        right += 1
    confusion_matrix[targets[i]][prediction] += 1
    #else:
    #    print('TARGET:',targets[i])
    #    print('distance spring:',dist_spring)
    #    print('distance summer:',dist_summer)
    #    print('distance autumn:', dist_autumn)
    #    print('distance winter:',dist_winter)

In [40]:
print(right)
print('accuracy:', right/len(targets))

print(confusion_matrix)

727
accuracy: 0.798901098901099
[[155.  46.   4.  41.]
 [ 10. 212.   0.  10.]
 [ 17.  17. 168.   6.]
 [ 26.   4.   2. 192.]]


In [43]:
#5-ppv
import numpy as np
coef = [0.7, 0.3]
confusion_matrix = np.zeros((4,4))
right = 0
for conf in configuration:
    test = conf[0][0]
    train = conf[1]
    
    min_dist = [1000, 1000, 1000, 1000, 1000]
    min_index = [-1, -1, -1, -1, -1]
    
    for sample in train:
            dist = tu.distance(signatures[test], signatures[sample], coef)
            index_min_element = min_dist.index(max(min_dist))
            if(dist < min_dist[index_min_element]):
                min_dist[index_min_element] = dist
                min_index[index_min_element] = sample
    
    index_most_common = min_dist.index(most_frequent(min_dist))
    minimun_index = min_index[index_most_common]
    prediction = targets[minimun_index]
    
    if(prediction == targets[test]):
        right += 1
    confusion_matrix[targets[test]][prediction] += 1

In [45]:
print(right)
print('accuracy:', right/len(targets))

print(confusion_matrix)

718
accuracy: 0.789010989010989
[[168.  37.   7.  34.]
 [ 29. 190.   6.   7.]
 [ 12.  10. 176.  10.]
 [ 31.   5.   4. 184.]]


In [42]:
 
from collections import Counter 
  
def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0] 
    
List = [2, 1, 2, 2, 1, 3] 
print(most_frequent(List)) 
print(List.index(2))

2
0


In [5]:
right = 0
coef = [0.8, 0.2]
for i,sig_1 in enumerate(signatures):
    d_min = 10000
    index = -1
    for j,sig in enumerate(signatures):
        if(i == j):
            continue
        else:
            dist = tu.distance(sig_1, sig, coef)
            #prediction spring = 0, summer = 1, autumn = 2, winter = 3
            if(dist < d_min):
                d_min = dist
                index = j

    #check if prediction was good
    prediction = targets[index]
    if(prediction == targets[i]):
        right += 1
    else:
        print('TARGET:',targets[i])
        #print('distance spring:',dist_spring)
        #print('distance summer:',dist_summer)
        #print('distance autumn:', dist_autumn)
        #print('distance winter:',dist_winter)
        

print(right)
print('accuracy:', right/len(targets))

TARGET: 0
TARGET: 1
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 1
TARGET: 1
TARGET: 1
TARGET: 1
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 1
TARGET: 1
TARGET: 3
TARGET: 1
TARGET: 0
TARGET: 1
TARGET: 0
TARGET: 3
TARGET: 3
TARGET: 3
TARGET: 3
TARGET: 1
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 2
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 2
TARGET: 0
TARGET: 0
TARGET: 2
TARGET: 0
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 1
TARGET: 0
TARGET: 1
TARGET: 2
TARGET: 3
TARGET: 3
TARGET: 1
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 1
TARGET: 1
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 1
TARGET: 2
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 3
TARGET: 2
TARGET: 1
TARGET: 1
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 1
TARGET: 1
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 0
TARGET: 0
TARGET: 0
TARGET: 3
TARGET: 3
TARGET: 1


In [33]:
print('Reference:',filenames[0])

print('Winter test:',filenames[1])
print('Spring test:',filenames[2])
print('Summer test:',filenames[3])
print('Autumn test:',filenames[4])

#import tree_utils as tu
dist_winter = tu.distance(centroid_winter, signatures[1], [0.8, 0.2])
print('Distante to a winter:',dist)
dist_spring = tu.distance(centroid_spring, signatures[1], [0.8, 0.2])
print('Distante to a spring:',dist)
dist_summer = tu.distance(centroid_summer, signatures[1], [0.8, 0.2])
print('Distante to a summer:',dist)
dist_autumn = tu.distance(centroid_autumn, signatures[1], [0.8, 0.2])
print('Distante to a autumn:',dist)
if(dist_spring <= dist_winter and dist_spring <= dist_autumn and dist_spring <= dist_summer):
    prediction = 0
elif(dist_summer <= dist_winter and dist_summer <= dist_spring and dist_summer <= dist_autumn):
    prediction = 1
elif(dist_autumn <= dist_winter and dist_autumn <= dist_spring and dist_autumn <= dist_summer):
    prediction = 2
else:
    prediction = 3

print(prediction)
print(targets[1])

if(prediction == int(targets[1])):
    right += 1

print(right)

Reference: ../FruitLearning/Resources/database/spring/137.png
Winter test: ../FruitLearning/Resources/database/winter/726.png
Spring test: ../FruitLearning/Resources/database/spring/175.png
Summer test: ../FruitLearning/Resources/database/summer/406.png
Autumn test: ../FruitLearning/Resources/database/autumn/622.png
Distante to a winter: 0.5850568501035276
Distante to a spring: 0.5850568501035276
Distante to a summer: 0.5850568501035276
Distante to a autumn: 0.5850568501035276
3
3
663


In [62]:
dist = tu.distance(centroid_spring, signatures[1], [0.8, 0.2])
print('Distante to a winter:',dist)
dist = tu.distance(centroid_spring, signatures[2], [0.8, 0.2])
print('Distante to a spring:',dist)
dist = tu.distance(centroid_spring, signatures[3], [0.8, 0.2])
print('Distante to a summer:',dist)
dist = tu.distance(centroid_spring, signatures[4], [0.8, 0.2])
print('Distante to a autumn:',dist)

Distante to a winter: 0.43050714858488
Distante to a spring: 0.3094622777727072
Distante to a summer: 0.39922652928334057
Distante to a autumn: 0.6052329905561256


In [14]:
from keras.preprocessing import image
img = cv2.imread('../FruitLearning/Resources/database/autumn/511.jpg')
if(img is None):
    print("noen")
#tensor_3d = image.img_to_array(img)
#tensor_3d.shape